In [1]:
%matplotlib inline
import os,sympy, csv,pickle,random

import matplotlib.pyplot as plt
import scipy.io as sio
from scipy.optimize import linprog
from scipy.io import loadmat
from scipy.linalg import *
from numpy import *
from math import sqrt
from time import time
from scipy.io import loadmat,savemat
from random import shuffle
from sklearn import tree
from sympy import Matrix

In [16]:
def loadData(name,path='.'):
    '''
    This loads a pickle file and returns the content which is a DICTIONARY object in our case.
    '''
    if ".pkl" in name:
            name = name.split(".pkl")[0]
    if "/" in name:
            name = name.split("/",1)[1]

    with open(path+"/"+name + '.pkl', 'rb') as f:
          return pickle.load(f)

def saveData(obj, name,path='.'):
    '''
    This saves a object into a pickle file. In our case, it is generally a DICTIONARY object.
    '''

    with open(path+"/"+name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

## Loading the data 

In [31]:
compliance =loadmat('compliance.mat')['compliance']
len(compliance),len(compliance[0])

(36, 76)

In [32]:
vf =loadmat('volumefraction.mat')['volumefraction']
constraints = sio.loadmat('constraints.mat')
stiffness = constraints['stiffness']

## Constructing C

In [33]:
C = []
C += [vf[0]]

In [34]:
C +=[compliance[0]]
C +=[compliance[35]]

In [35]:
f1=loadmat('linprog_a_yield.mat')['y']

C+= [f1.transpose()[0]]

In [134]:
Cmatrix = {}
Cmatrix['C'] = C
savemat("Cmatrix",Cmatrix)

In [84]:
len(compliance)

36

### Loading the best solution

In [36]:
odfHigher_1 = loadData('odfBestLower_1')

### Taking nullspace of the matrix

In [166]:
nullCMatrix = {}
nullCMatrix['nullC']= V
savemat('nullCMatrix',nullCMatrix)

In [37]:
nullC = loadmat('nullC')['nullC']

In [41]:
nullCTrans = nullC.transpose()

## <font color="red">Extracting the vectors from nullspace matrix</font>

In [43]:
V =[]
for i in range(len(nullCTrans)):
    V += [squeeze(asarray(nullCTrans[i]))]#.astype(float)]

In [16]:
def posPositive(elems):
    positive = []
    for i in range(len(elems)):
        elem = elems[i]
        if elem>0:
            positive += [i]
    return positive

In [15]:
def posNegative(elems):
    negative = []
    for i in range(len(elems)):
        elem = elems[i]
        if elem<0:
            negative += [i]
    return negative

In [10]:
def allPositive(elems):
    for elem in elems:
        if elem<0:
            return False
    return True

In [11]:
def allNegative(elems):
    for elem in elems:
        if elem>0:
            return False
    return True

In [28]:
x = [1,-1,2,3]
y = [1,-2,3,4]
print posPositive(x)
print posPositive(y)
if posPositive(x)==posPositive(y):
    print 'yes'

[0, 2, 3]
[0, 2, 3]
yes


In [29]:
for i in range(len(V)-1):
    for j in range(i+1,len(V)):
        if posPositive(V[i]) == posPositive(V[j]):
            print i,j,'positive'
        if posNegative(V[i]) == posNegative(V[j]):
            print i,j,'negative'
        if (posNegative(V[i]) == posPositive(V[j])) or (posNegative(V[j]) == posPositive(V[i])):
            print i,j,'alternate'
            

In [44]:
len(V),len(V[0])

(72, 76)

In [45]:
altOdf = odfHigher_1 + V[0]

In [46]:
def isValid(odfVector):
    for odf in odfVector:
        if odf <0:
            return False
    return True

In [47]:
for i in range(72):
    for j in range(100):
        val = random.uniform(0, 10)
        altOdf = odfHigher_1 + (val*V[i])
        if isValid(altOdf):
            print i,val

In [48]:
array = []
for i in range(72):
    d = []
    for j in range(76):
        if V[i][j] == 0:
            pass
        else:
            d += [- odfHigher_1[i]/V[i][j]]
    array += [d]

In [103]:
for i in range(72):
   
    val = 2.72644284543468
    altOdf = odfHigher_1 + (val*V[i])
#     if isValid(altOdf):
    print i,altOdf

0 [0 -2.72644284543468 2.72644284543468 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 67.8061980717923 0 0 0 0 0 0 0 0 0 0 0 0
 2.33023277474475 0 0 0 0 25.8676435594358 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 154.925832712635]
1 [-0.212536237845949 0.415168250363609 0 -0.259688161375865
 -2.46675468405882 2.72644284543468 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 67.8061980717923 0 0 0 0 0 0 0 0 0 0 0 0 2.33023277474475 0 0
 0 0 25.8676435594358 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 154.925832712635]
2 [6.91288971342035e-16 -1.78212457520662e-15 0 -2.72644284543468
 -8.44653898867562e-16 0 2.72644284543468 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 67.8061980717923 0 0 0 0 0 0 0 0 0 0 0 0 2.33023277474475
 0 0 0 0 25.8676435594358 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 154.925832712635]
3 [-0.0436000557257055 0.0610056140427951 0 -2.70364439680268
 0.682532915622238 0 0 2.72644284543468 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 

In [112]:
my_list = [0,1,-1,-2,3]
List = nonzero(odfHigher_1)[0]
List

array([32, 45, 50, 75])

In [118]:
for i in range(72):
    print V[i]
#     elems = nonzero(V[i])[0]
#     for elem in elems:
#         if elem in List:
#             print i,V[i],elem

[0 -1.00000000000000 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0]
[-0.0779536744009991 0.152274694134444 0 -0.0952479755116459
 -0.904752024488354 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
[2.53549775488443e-16 -6.53644575088277e-16 0 -1.00000000000000
 -3.09800698841680e-16 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0]
[-0.0159915531692557 0.0223755338003679 0 -0.991638024369307
 0.250338244487726 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
[-0.123951181088390 0.190620775271367 0 -1.01098256805490 0.362585123740386
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [8]:
nullC = loadmat('nullC.mat')['nullC']
len(nullC),len(nullC[0])

(76, 72)

In [11]:
negNullC = -nullC

In [12]:
negNullC[0]

array([ 0.21312224,  0.23245002,  0.12641739,  0.08007532,  0.10839263,
        0.05228106,  0.10264048, -0.01063165, -0.08129722,  0.04008383,
       -0.02364195,  0.05223759,  0.08181522,  0.03016027, -0.03991011,
       -0.08093279, -0.04451283,  0.07789006, -0.04141037,  0.02984172,
        0.10832795,  0.09522992,  0.02906058, -0.02658704,  0.02077937,
        0.0382937 ,  0.08625335,  0.04883413,  0.00460459,  0.07303757,
       -0.04475784, -0.02536029, -0.04723089, -0.02988687, -0.04475784,
       -0.02766998,  0.02120472, -0.03603318, -0.02536029, -0.02988687,
        0.00204332, -0.03624474,  0.03713359, -0.03624474,  0.03713359,
        0.01806129,  0.00204332, -0.02766998,  0.00447528, -0.00056648,
        0.00447528, -0.07035196, -0.00056648,  0.02120472, -0.07153569,
       -0.01083319, -0.02303243, -0.01554891, -0.09229752, -0.16234168,
       -0.01032799, -0.01502538, -0.00964574, -0.09872901, -0.0594675 ,
       -0.1692993 , -0.0559659 , -0.06554902, -0.03174466, -0.05

In [76]:
b

array([   0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,   83.65844314,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.     

In [74]:
c = [0]*72
c[0] = 1
c[45] = 0.1
c[71] = -1
b = loadmat('maxOdf.mat')['odf'][0]
A = negNullC
xBounds = []
for i in range(2):
    xBounds += [(None,None)]
for i in range(70):
    xBounds += [(None,None)]
res = linprog(c, A_ub=A, b_ub=b, bounds=tuple(xBounds),options={"disp": True})

Optimization terminated successfully.
         Current function value: -11.679870  
         Iterations: 98


In [78]:
res['slack']

array([   0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.27551368,    0.        ,
         67.14043253,   21.03937637,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.     

In [57]:
tuple(xBounds)

((0.001, None),
 (0.001, None),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 0.001),
 (None, 

In [37]:
c = [-1, 4]
A = [[-3, 1], [1, 2]]
b = [6, 4]
# x0_bounds = (None, None)
x1_bounds = (-3, None)
res = linprog(c, A_ub=A, b_ub=b, bounds=(x1_bounds),options={"disp": True})
res

Optimization terminated successfully.
         Current function value: -22.000000  
         Iterations: 1


     fun: -22.0
 message: 'Optimization terminated successfully.'
     nit: 1
   slack: array([ 39.,   0.])
  status: 0
 success: True
       x: array([ 10.,  -3.])

In [71]:
Lambda = res['x']
B = nullC.dot(Lambda)
A = loadmat('maxOdf.mat')['odf'][0]
A+B

array([  1.47116659e-14,  -3.86975779e-13,   1.37639253e-13,
        -4.93636510e-12,  -9.27470002e-14,  -3.36100935e-13,
        -6.71075900e-14,   5.66702475e-14,   2.40640333e-12,
         1.62768557e-13,   6.42138932e-13,   5.07064180e-13,
        -1.25544734e-13,  -5.36503770e-14,   2.67076657e-13,
         6.60483043e-13,  -1.28304593e-14,   3.63120143e-13,
        -1.46702824e-13,   1.74260044e-13,  -9.76741254e-14,
         5.74655414e-13,   1.86699261e-13,   5.46756225e-13,
         7.01215792e-13,   4.93181246e-13,   1.52321015e-13,
         2.02625361e-13,   4.39693058e-13,   2.05266545e-13,
         8.67012538e-13,   8.93152586e-13,   6.52216002e+01,
         2.13373142e+01,   2.20582632e-13,  -1.74918782e-13,
         1.91689213e-13,  -1.25744947e-13,   3.20058615e-13,
         9.60533035e-14,   2.36226805e-13,   3.12181866e-13,
        -8.96536539e-14,   1.86893856e-13,   5.79794573e-14,
         2.80165568e-13,   5.39501428e-13,   3.08587277e-13,
         4.96868864e-13,

(2, 3, 1, 2)